In [1]:
%load_ext tensorboard
import tensorflow as tf
import sklearn as sk 
import matplotlib.pyplot as plt
import cv2 as cv 
import keras 
import os 
import pandas as pd
from PIL import Image
import itertools 
from sklearn.datasets import *
from sklearn.metrics import confusion_matrix
from keras.utils import plot_model 
import random 
from keras.layers import *
import numpy as np
from datasets import Dataset
import datetime

In [2]:
tensor = {
    "train": ...,
    "test": ..., 
    "valid": ..., 
} 
path = {
    "train":".\\data\\imgs\\ASL_Alphabet\\train",
    "test":".\\data\\imgs\\ASL_Alphabet\\test", 
    "valid":".\\data\\imgs\\ASL_Alphabet\\valid" 
} 
classes = [] 

In [3]:
image_height = 64
image_width = 64
image_size = (image_height,image_width)
batch_size = 8
train_size = 10000
val_size = int(train_size*0.2)

In [4]:
def create_dataset(directory):
    return keras.preprocessing.image_dataset_from_directory(
        directory,
        image_size=(image_height, image_width),
        batch_size=batch_size,
        color_mode='rgb',
        label_mode='categorical' if directory != path["test"] else None,
        shuffle=True if directory == path["train"] else False,
        seed=42,
    )

In [5]:
test_filenames = os.listdir(path["test"])
test_df = pd.DataFrame({
    'filename': test_filenames
})

In [6]:
train_dataset = create_dataset(path["train"])
valid_dataset = create_dataset(path["valid"])
train_dataset = train_dataset.prefetch(buffer_size=tf.data.AUTOTUNE)
valid_dataset = valid_dataset.prefetch(buffer_size=tf.data.AUTOTUNE)
test_dataset = Dataset.from_pandas(test_df)

Found 87000 files belonging to 29 classes.
Found 870 files belonging to 29 classes.


In [7]:
train_dataset

<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 64, 64, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 29), dtype=tf.float32, name=None))>

In [8]:
tensor["train"] = train_dataset
tensor["valid"] = valid_dataset
tensor["test"] = test_dataset

In [9]:
train = list(tensor["train"])
valid = list(tensor["valid"])

In [10]:
def convert(data: list[tuple[tf.Tensor,tf.Tensor]]):
    nparr = []
    nplabel = []
    for i in data:
        for x in i[0]:
            nparr.append(tf.squeeze(x).numpy())
        for y in i[1]:
            nplabel.append(y.numpy())
    arr = np.array(nparr)
    labels = np.array(nplabel)
    return arr, labels

In [11]:
train_data,train_labels = convert(train)
valid_data,valid_labels = convert(valid)

In [12]:
train_data.shape,train_labels.shape, valid_data.shape,valid_labels.shape

((87000, 64, 64, 3), (87000, 29), (870, 64, 64, 3), (870, 29))

In [13]:
norm_train=train_data[:train_size]
norm_valid=valid_data[:val_size]
norm_train.shape,norm_valid.shape

((10000, 64, 64, 3), (870, 64, 64, 3))

In [14]:
int_train_labels = np.argmax(train_labels,axis=1)[:train_size]
int_valid_labels = np.argmax(valid_labels,axis=1)[:val_size]
int_train_labels.shape,int_valid_labels.shape

((10000,), (870,))

In [15]:
from keras.layers import *

def create_model(input_shape,num_classes):
    inputs = keras.Input(shape=input_shape)

    # Entry block
    x = Rescaling(1.0 / 255)(inputs)
    x = Conv2D(128, 3, strides=2, padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    previous_block_activation = x  # Set aside residual

    for size in [256, 512, 728]:
        x = Activation("relu")(x)
        x = SeparableConv2D(size, 3, padding="same")(x)
        x = BatchNormalization()(x)

        x = Activation("relu")(x)
        x = SeparableConv2D(size, 3, padding="same")(x)
        x = BatchNormalization()(x)

        x = MaxPooling2D(3, strides=2, padding="same")(x)

        # Project residual
        residual = Conv2D(size, 1, strides=2, padding="same")(
            previous_block_activation
        )
        x = add([x, residual])  # Add back residual
        previous_block_activation = x  # Set aside next residual

    x = SeparableConv2D(1024, 3, padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    x = GlobalAveragePooling2D()(x)
    if num_classes == 2:
        units = 1
    else:
        units = num_classes

    x = Dropout(0.25)(x)
    # We specify activation=None so as to return logits
    outputs = Dense(units, activation=None)(x)
    return keras.Model(inputs, outputs)

In [16]:
!taskkill /IM tensorboard.exe /F
%tensorboard --logdir logs

ERROR: The process "tensorboard.exe" not found.


Reusing TensorBoard on port 6007 (pid 39324), started 17:01:00 ago. (Use '!kill 39324' to kill it.)

In [17]:
def train_model():
    model = create_model(input_shape=image_size + (3,), num_classes=29)
    keras.utils.plot_model(model, show_shapes=True)
    model.summary()
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.03),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'],)
    logdir = os.path.join(
        "logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
    tensorboard_callback = keras.callbacks.TensorBoard(
        logdir, histogram_freq=1)
    history = model.fit(norm_train,
                        int_train_labels,
                        validation_data=(
                            norm_valid,
                            int_valid_labels
                        ),
                        epochs=100,
                        callbacks=[tensorboard_callback])
    return history, model

In [18]:
history, model = train_model()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 64, 64, 3) │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling           │ (None, 64, 64, 3) │          0 │ input_layer[0][0] │
│ (Rescaling)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 32, 32,    │      3,584 │ rescaling[0][0]   │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 32, 32,    │        512 │ conv2d[0][0]      │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation          │ (None, 32, 32,    │          0 │ batch_normalizat… │
│ (Activation)        │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_1        │ (None, 32, 32,    │          0 │ activation[0][0]  │
│ (Activation)        │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ separable_conv2d    │ (None, 32, 32,    │     34,176 │ activation_1[0][… │
│ (SeparableConv2D)   │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 32, 32,    │      1,024 │ separable_conv2d… │
│ (BatchNormalizatio… │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_2        │ (None, 32, 32,    │          0 │ batch_normalizat… │
│ (Activation)        │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ separable_conv2d_1  │ (None, 32, 32,    │     68,096 │ activation_2[0][… │
│ (SeparableConv2D)   │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 32, 32,    │      1,024 │ separable_conv2d… │
│ (BatchNormalizatio… │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d       │ (None, 16, 16,    │          0 │ batch_normalizat… │
│ (MaxPooling2D)      │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 16, 16,    │     33,024 │ activation[0][0]  │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add (Add)           │ (None, 16, 16,    │          0 │ max_pooling2d[0]… │
│                     │ 256)              │            │ conv2d_1[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_3        │ (None, 16, 16,    │          0 │ add[0][0]         │
│ (Activation)        │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ separable_conv2d_2  │ (None, 16, 16,    │    133,888 │ activation_3[0][… │
│ (SeparableConv2D)   │ 512)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 16, 16,    │      2,048 │ separable_conv2d

 Total params: 2,759,765 (10.53 MB)

 Trainable params: 2,751,477 (10.50 MB)

 Non-trainable params: 8,288 (32.38 KB)

Epoch 1/100


In [ ]:
pd.DataFrame(history.history)["learning_rate"].plot()
pd.DataFrame(history.history)["accuracy"].plot()
plt.legend()
plt.grid(True)

In [ ]:
pd.DataFrame(history.history)["loss"].plot()
pd.DataFrame(history.history)["accuracy"].plot()
plt.legend()
plt.grid(True)

In [ ]:
pd.DataFrame(history.history)["val_loss"].plot()
pd.DataFrame(history.history)["val_accuracy"].plot()
plt.legend()
plt.grid(True)

In [ ]:
test_set = [tf.convert_to_tensor(plt.imread(os.path.join(path["test"],i))) for i in os.listdir(path["test"])]
test_set = tf.cast(test_set,dtype=tf.float32)/255.0
test_set[0].shape

In [ ]:
pred = model.predict(test_set)
pred

In [ ]:
test_imgs = os.listdir("./data/imgs/compress/test/")
test_img, test_labels = [],[]
c = 0
for file in test_imgs:
    im = tf.io.read_file(os.path.join("./data/imgs/compress/test/",file))
    im = tf.image.decode_image(im)
    im = tf.cast(im,dtype=tf.float32)
    im = im/255.0
    test_img.append(im)
    temp_label = [0.0]*29
    temp_label[c] = 1.0
    c += 1
    test_labels.append(tf.convert_to_tensor(temp_label))
test_img = tf.convert_to_tensor(test_img)
test_labels = tf.convert_to_tensor(test_labels).numpy()

In [ ]:
test_labels.shape, pred.shape, pred[:2],test_labels[:2]

In [ ]:
y_test = test_labels.flatten()
y_pred = pred.flatten()
y_test,y_pred

In [ ]:
y_x = np.array([i for i in range(1,len(y_test)+1)])
y_x

In [ ]:
plt.scatter(y_x,y_test,label="Tested")
plt.scatter(y_x,y_pred,label="Predicted")
plt.grid(True)
plt.legend()
plt.show()

In [ ]:
y_pred = np.argmax(pred,axis=1)
y_test = np.argmax(test_labels,axis=1)
y_x = np.array([i for i in range(0,29)])
y_pred,y_test,y_x

In [ ]:
plt.scatter(y_x,y_test,label="Tested")
plt.scatter(y_x,y_pred,label="Predicted")
plt.grid(True)
plt.legend()
plt.show()